In [29]:
%load_ext autoreload 
%autoreload 2
%matplotlib inline
import numpy as np
import sympy
import scipy.sparse.linalg
import scipy.sparse as sps
from math import pi

import setup_grids
from core.constit import second_order_tensor
from core.bc import bc
from fvdiscr import tpfa, fvutils
from viz import cell_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# Homogeneous media

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 5
grid_type = 'cart'
pert = 0.5
### End of parameter definitions

# Permeability tensor, scalar for simplicity
perm = 1

# Analytical solution
x, y = sympy.symbols('x y')
u = sympy.sin(x) * sympy.cos(y)
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')


def run_convergence(grid_type):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        k = second_order_tensor.SecondOrderTensor(2, perm * np.ones(g.num_cells))

        # Set type of boundary conditions - Dirichlet
        bound_faces = g.get_boundary_faces()
        bound_cond = bc.BoundaryCondition(g, bound_faces, ['dir'] * bound_faces.size)
        
        # MPFA discretization, and system matrix
        flux, bound_flux = tpfa.tpfa(g, k, bound_cond)
        div = fvutils.scalar_divergence(g)
        a = div * flux
        
        # Boundary conditions
        xf = g.face_centers
        u_bound = np.zeros(g.num_faces)
        u_bound[bound_faces] = u_f(xf[0, bound_faces], xf[1, bound_faces])
        
        # Right hand side - contribution from the solution and the boundary conditions
        xc = g.cell_centers
        b = rhs_f(xc[0], xc[1]) * g.cell_volumes + div * bound_flux * u_bound

        # Solve system, derive fluxes
        u_num = scipy.sparse.linalg.spsolve(a, b)
        flux_num = flux * u_num - bound_flux * u_bound

        # Exact solution
        u_ex = u_f(xc[0], xc[1])
        du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
        flux_ex = -np.sum(g.face_normals[:2] * du_ex_faces, axis=0)
        flux_diff = flux_num - flux_ex
  
        u_err.append(np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

grids = ['cart', 'triangular']

for gr in grids:
    u, f = run_convergence(gr)
    print('Grid Type: ',gr)
    print(u, 'pressure error')
    print(f, 'flux error')

Grid Type:  cart
[0.026359702657823614, 0.012578154046047901, 0.0074035828559291259, 0.0039120630419055029, 0.002725997562087335] pressure error
[0.22312616554604783, 0.1382281114291416, 0.14268545431544949, 0.1513841330607931, 0.14817483610585283] flux error
Grid Type:  triangular
[0.054329906930995056, 0.022328117107637782, 0.018154990177680699, 0.014884044910978193, 0.014118688534210557] pressure error
[0.364296991122602, 0.41420476072677698, 0.40968572887823723, 0.40826113298585942, 0.40450191203098734] flux error


In [43]:
# Neuman boundary, homogeneous media

###
# This is where parameters can be modified to alter the convergence test.
# The remaining lines should 
np.random.seed(42)
base = 4
domain = np.array([1, 1])
basedim = np.array([base, base])
num_refs = 7
grid_type = 'cart'
pert = 0
### End of parameter definitions

# Permeability tensor, scalar for simplicity
perm = 1

# Analytical solution
x, y = sympy.symbols('x y')
u = sympy.sin(x) * sympy.cos(y)
u_f = sympy.lambdify((x, y), u, 'numpy')
dux = sympy.diff(u, x)
duy = sympy.diff(u, y)
dux_f = sympy.lambdify((x, y), dux, 'numpy')
duy_f = sympy.lambdify((x, y), duy, 'numpy')
rhs = -sympy.diff(dux, x) - sympy.diff(duy, y)
rhs_f = sympy.lambdify((x, y), rhs, 'numpy')


def run_convergence(grid_type):
    u_err = []
    flux_err = []

    for g in setup_grids.grid_sequence(basedim, num_refs, grid_type, pert):
        # Reset the random seed for every grid realization.
        # This should make no difference for the convergence test, 
        # but it makes sure that we can run unit tests based on the values obtained
        # here.
        np.random.seed(42)
        
        # Permeability tensor
        k = second_order_tensor.SecondOrderTensor(2, perm * np.ones(g.num_cells))
        
        # Exact solution
        xf = g.face_centers
        xc = g.cell_centers
        u_ex = u_f(xc[0], xc[1])
        du_ex_faces = np.vstack((dux_f(xf[0], xf[1]), duy_f(xf[0], xf[1])))
        if du_ex_faces.shape[1] != xf.shape[1]:
            assert du_ex_faces.shape[1] == 1
            du_ex_faces = np.matlib.repmat(du_ex_faces, 1, xf.shape[1])
        n_faces = g.face_normals[:2] / np.sqrt(np.sum(g.face_normals**2,axis=0))
        flux_ex = -np.sum(n_faces * du_ex_faces, axis=0)
        
        # Set type of boundary conditions - Dirichlet+
        n = np.max(g.nodes)
        top = np.ravel(np.argwhere(g.face_centers[1, :] > n - 1e-10))
        bot = np.ravel(np.argwhere(g.face_centers[1, :] < 1e-10))
        left = np.ravel(np.argwhere(g.face_centers[0, :] < 1e-10))
        right = np.ravel(np.argwhere(g.face_centers[0, :] > n - 1e-10))

        dir_faces = top
        neu_faces = np.hstack((left,right,bot))
        bound_cond = bc.BoundaryCondition(g, dir_faces, ['dir'] * dir_faces.size)
        
        # TPFA discretization, and system matrix
        flux, bound_flux = tpfa.tpfa(g, k, bound_cond)
        div = fvutils.scalar_divergence(g)
        a = div * flux
        
        # Boundary conditions
        nfi, _, sgn = sps.find(g.cell_faces[neu_faces,:])
        u_bound = np.zeros(g.num_faces)
        u_bound[dir_faces] = u_f(xf[0, dir_faces], xf[1, dir_faces])
        u_bound[neu_faces[nfi]] = flux_ex[neu_faces[nfi]]*(-sgn)

        # Right hand side - contribution from the solution and the boundary conditions
        b = rhs_f(xc[0], xc[1]) * g.cell_volumes + div * bound_flux * u_bound

        # Solve system, derive fluxes
        u_num = scipy.sparse.linalg.spsolve(a, b)
        flux_num = (flux * u_num - bound_flux * u_bound) / g.face_areas
        # Error estimates
        flux_diff = flux_num - flux_ex
  
        u_err.append(np.sqrt(np.sum(g.cell_volumes * (u_num - u_ex)**2)) /
                     np.sqrt(np.sum(g.cell_volumes * u_ex**2)))
        flux_err.append(np.sqrt(np.sum((g.face_areas ** g.dim) * flux_diff**2))/
                        np.sqrt(np.sum((g.face_areas ** g.dim) * flux_ex**2)))
    return u_err, flux_err

grids = ['cart', 'triangular']

for gr in grids:
    u, f = run_convergence(gr)
    print('Grid: ', gr)
    print(u,'pressure error')
    print(f,'flux error')


Grid:  cart
[0.0072022282476458871, 0.0017874530851249273, 0.0004460699832864939, 0.00011146844452597047, 2.7864055519112392e-05, 6.9658229623538602e-06, 1.7414437345009914e-06] pressure error
[0.0022799310895371587, 0.0005807385581614021, 0.00014548172425695252, 3.6307175629253304e-05, 9.0606912100608403e-06, 2.2625436728961341e-06, 5.6526299621196908e-07] flux error
Grid:  triangular
[0.33622202791506212, 0.32945417281790706, 0.32745554253404591, 0.32686122259359407, 0.32668333338039296, 0.32662978256307185, 0.32661358163812665] pressure error
[0.10387751757535411, 0.10979724829016586, 0.11083866335913663, 0.11066898722505635, 0.11035446809747386, 0.11012471086881058, 0.10998779722025241] flux error
